# Introduction on data

In this exercise we combine issuances and holdings datasets, and we fetch legal entities Level 2 data using [GLEIF API](https://documenter.getpostman.com/view/7679680/SVYrrxuU?version=latest#40ef2ec4-b8bd-46de-8ad5-5359ed828242), to connect corporate dots and simulate a financial markets analysis on defaults. 

<u>NOTE</u>: The provided granular datasets *issuances.csv* and *holdings.csv* are composed of artificial data: the ISINs in the first datasets were randomly generated, and instruments were casually assigned to holder companies. We are pretending to own two datasets providing granular securities data on issuances and holdings perspective.

In [2]:
import pandas as pd

In [3]:
issuances_df = pd.read_csv('issuances.csv', sep=';')
holdings_df = pd.read_csv('holdings.csv')

*issuances.csv* provides ISIN-by-ISIN information, i.e. for each security the following details are provided:
- *isin*: the ID of the security;
- *issue_date*: date of issuance;
- *publication_price*: price of publication;
- *volume*: number of outstanding amount of stock;
- *market_capitalization*: market value of the company's outstanding amount of stock;
- *issuer_lei*: the ID (LEI) of the issuer company.

The Legal Entity Identifier (LEI) is a 20-character, alpha-numeric code based on the ISO 17442 standard developed by the International Organization for Standardization (ISO). [1]()

In [4]:
issuances_df.head()

,isin,issue_date,publication_price,volume,market_capitalization,issuer_lei
0,BY8418434092,02/08/2023,108.00,2259,243972.0,5493004QI6E3PJNCEB09
1,MK5864929728,13/02/2023,100.00,10,1000.0,635400SCBVV58JBVY513
2,EE6894244009,12/06/2020,98.00,2091,204918.0,549300PBHN7G23NA7J97
3,DK7859950600,05/01/2023,0.25,1020,255.0,5299000AF58N6DVTY088
4,IT6091559071,02/02/2022,53.00,2001,106053.0,549300MF6CNUB2URPE58


*holdingd.csv* provides on information about who is holding what security:
- *isin*: the ID of the security;
- *holder_lei*: the ID (LEI) of the holder company

In [5]:
holdings_df.head()

,isin,holder_lei
0,SK4383059521,254900RGIHNSOOTM0L46
1,SK4383059521,2138006HP23N8GYS9Q88
2,SK4383059521,529900JYUND014UQ0P58
3,SK4383059521,549300M7R6X5LJOH0491
4,CH8229968527,254900WJJF84AQA4EN11


The two datasets point to two different aspects of financial market, and if we join them, we can have a broader perspective on the market.

If we can merge them based on the *ISIN*, we can observe if from securities point of view: given one security, we can find information on the issuer and the holder of that security. 

We could also join based on company (i.e. *issuer_lei* on *holder_lei*) to see whether an issuer company is also a holder of securities issued by another company.

## Companies data

### Level 1 Data

While various instrument details are available, no details on companies were provided. To collect them, we can use [GLEIF API](https://documenter.getpostman.com/view/7679680/SVYrrxuU?version=latest#40ef2ec4-b8bd-46de-8ad5-5359ed828242).

The GLEIF API gives developers access to full LEI Data search engine functionality, including filters, full-text and single-field searches of Level 1 (LEI Record) Data, retrieval of LEI Records (including links to their Level 2 data, where available), based on a search of their associated Level 2 (relationship) data, and "fuzzy" matching of important data fields such as names and addresses.

Requests are HTTP REST calls, following the JSON API specification. [2]()

In [6]:
import requests

To get the entity details we are looking for, we just need the LEI of the company.

In [7]:
session = requests.Session()
url = "https://api.gleif.org/api/v1/lei-records/"

In [8]:
session_json = session.get(url , headers={"Accept": "application/vnd.api+json"})

In [9]:
session_json.json()

{'meta': {'goldenCopy': {'publishDate': '2024-02-26T08:00:00Z'},
  'pagination': {'currentPage': 1,
   'perPage': 15,
   'from': 1,
   'to': 15,
   'total': 2571457,
   'lastPage': 171431}},
 'links': {'first': 'https://api.gleif.org/api/v1/lei-records?page%5Bnumber%5D=1&page%5Bsize%5D=15',
  'next': 'https://api.gleif.org/api/v1/lei-records?page%5Bnumber%5D=2&page%5Bsize%5D=15',
  'last': 'https://api.gleif.org/api/v1/lei-records?page%5Bnumber%5D=171431&page%5Bsize%5D=15'},
 'data': [{'type': 'lei-records',
   'id': '335800GZE1G6HXO9W331',
   'attributes': {'lei': '335800GZE1G6HXO9W331',
    'entity': {'legalName': {'name': 'SUNRAYS INDIA', 'language': 'en'},
     'otherNames': [],
     'transliteratedOtherNames': [],
     'legalAddress': {'language': 'en',
      'addressLines': ['PLOT NO.7310/4, GIDC SACHIN, ROAD NO.73B, SACHIN,'],
      'addressNumber': None,
      'addressNumberWithinBuilding': None,
      'mailRouting': None,
      'city': 'SURAT',
      'region': 'IN-GJ',
      '

Let's suppose we want to know the <b> name</b> and <b>legal address country</b> of one company, for example *Intesa San Paolo SPA*. First we need a parameter to use for the searching: the best choice is to use LEI, since is represents an unique identifier. If we do not know the LEI of a specific entity, we can use the [LEI search functionality of GLEIF](https://search.gleif.org/#/search/).

Information can be scraped in two ways: calling node by node in the json structure, or referencing to the specific LEI directly in the URL. In this case we can call the specific entity by adding a new component to the base URL, i.e. the LEI "2W8N8UU78PMDQKZENC08".

In [12]:
url_isp = url + "8156008F4B12928FDE93"
session_response = session.get(url_isp, headers={"Accept": "application/vnd.api+json"})

In [13]:
session_response.json()["data"]["attributes"]["entity"]["legalName"]["name"]

'FAI PROGETTO ITALIA II'

In [14]:
session_response.json()["data"]["attributes"]["entity"]["legalAddress"]["country"]

'IT'

### Level 2 Data

When working with finacial corporate data, it is necessary also to piece together the "corporate dots", i.e. take into consideration

LEI data pool includes also the "Level 2" data that answers the question of "who owns whom?". Specifically, legal entities that have or acquire an LEI report their "direct accounting consolidating parent" as well as their "ultimate accounting consolidating parent". [3]()

<img src="images/level-2-parents-and-children.png" alt="Drawing" style="width: 400px;"/>

To retreive data of direct or ultimate relationships (i.e. parent or child), you can enrich the url used in the get request.

In [17]:
company_lei = "2W8N8UU78PMDQKZENC08"
direct_childen = url + company_lei + "/direct-children"
ultimate_childen = url + company_lei + "/ultimate-children"

In [18]:
session_response_direct_children = session.get(direct_childen, headers={"Accept": "application/vnd.api+json"})
session_response_direct_children.json()

{'meta': {'goldenCopy': {'publishDate': '2024-02-26T08:00:00Z'},
  'pagination': {'currentPage': 1,
   'perPage': 15,
   'from': 1,
   'to': 15,
   'total': 31,
   'lastPage': 3}},
 'links': {'first': 'https://api.gleif.org/api/v1/lei-records/2W8N8UU78PMDQKZENC08/direct-children?page%5Bnumber%5D=1&page%5Bsize%5D=15',
  'next': 'https://api.gleif.org/api/v1/lei-records/2W8N8UU78PMDQKZENC08/direct-children?page%5Bnumber%5D=2&page%5Bsize%5D=15',
  'last': 'https://api.gleif.org/api/v1/lei-records/2W8N8UU78PMDQKZENC08/direct-children?page%5Bnumber%5D=3&page%5Bsize%5D=15'},
 'data': [{'type': 'lei-records',
   'id': '8156003F461C3D10B315',
   'attributes': {'lei': '8156003F461C3D10B315',
    'entity': {'legalName': {'name': 'ACANTUS S.P.A.', 'language': 'it-IT'},
     'otherNames': [],
     'transliteratedOtherNames': [],
     'legalAddress': {'language': 'it-IT',
      'addressLines': ['VIA GIAN DOMENICO ROMAGNOSI, 5'],
      'addressNumber': None,
      'addressNumberWithinBuilding': None

Let's suppose we want to know the <b>direct children</b> of company *Intesa San Paolo SPA* (we will ignore possible ultimate children for this example).

In [28]:
direct_children_isp = []
i=0
for item in session_response_direct_children.json()["data"]:
    i+=1
    item_lei = item['id']
    item_legal_name = item['attributes']['entity']['legalName']['name']
    direct_children_isp.append((i, item_lei, item_legal_name))

In [29]:
direct_children_isp

[(1, '8156003F461C3D10B315', 'ACANTUS S.P.A.'),
 (2, '815600EC178EEE69E481', 'EXETRA S.P.A.'),
 (3,
  '789000XDPJFDGEU86P31',
  'INTESA SANPAOLO SPA İTALYA İSTANBUL MERKEZ ŞUBESİ'),
 (4, '8156001ABCDB9E7C7705', 'PRESTITALIA S.P.A.'),
 (5, '222100VKE4U77H2IDP03', 'PRIVATE EQUITY INTERNATIONAL S.A.'),
 (6, '549300DR61OKD1UKCK14', 'FONDO PENSIONE DIPENDENTI EX CARIPRATO'),
 (7, '222100D7KSQ22ZVMKM20', 'GAP MANCO'),
 (8, '213800CBOERFQH934Q10', 'INTESA SANPAOLO BRASIL S.A. - BANCO MULTIPLO'),
 (9, '2221001BQYKCFUQGX885', 'INTESA SANPAOLO SEC SA'),
 (10, '222100ECVDB4RTLS8E02', 'IMI INVESTMENTS S.A.'),
 (11, '5493006UY56V4KORVP94', 'BANCA COMERCIALĂ “EXIMBANK” S.A.'),
 (12, '54930009CCENXT9WTF32', 'INTESA SANPAOLO BANK ALBANIA'),
 (13, '5493002D2LSSUA5KUB24', 'AlexBank'),
 (14, '5493001XF6EC2NJXK304', 'BANCA INTESA AKCIONARSKO DRUSTVO BEOGRAD'),
 (15,
  '815600D20361C46B7C65',
  'INTESA SANPAOLO RE.O.CO. S.P.A. O BREVEMENTE REOCO S.P.A.')]

The above result shows a list of 15 entities, but we know that *Intesa San Paolo SPA* has 31 direct children (see [here](https://search.gleif.org/#/record/2W8N8UU78PMDQKZENC08)). In fact, search results in GLEIF API response can be paginated, if the number of returned items is above 15, like in the presented case.

To handle the pagination, we could first analyse the structure of the response. "links" node contains "first", "next", and "last", corresponding to the links to first, last, and next page. When results are paginated, we would find them all, while you would see no "next" if no next page is available. Besides, also in "meta" node we can find information about pages, such how many pages are available for this request, and what number is the currently presented page.

Based on this, we could integrate the above loop with a condition interating on pages.

In [64]:
def parse_response(response):
    result = []
    for item in response.json()["data"]:
        id_item = item["id"]
        legal_name = item["attributes"]["entity"]["legalName"]["name"]
        legal_country = item["attributes"]["entity"]["legalAddress"]["country"]
        result.append((id_item, legal_name, legal_country))
    return result

In [65]:
def get_leis(parent_company, relationship):
    initial_url = url + parent_company + "/" + relationship
    response = session.get(initial_url, headers={"Accept": "application/vnd.api+json"})
    result = parse_response(response)
    while "next" in response.json()["links"]:
        next_url = response.json()["links"]["next"]
        response = session.get(next_url, headers={"Accept": "application/vnd.api+json"})
        next_result = parse_response(response=response)
        result.extend(next_result)

        if response.json()["meta"]["pagination"]["currentPage"] == response.json()["meta"]["pagination"]["lastPage"]:
            break
    return result

In [66]:
company_lei = "2W8N8UU78PMDQKZENC08"
direct_children = "direct-children"

In [68]:
len(get_leis(company_lei, direct_children))

31

## Exercise 1 

Retrieve the <b>legal address name</b>, the residence <b>country</b> and the <b>ultimate parent</b> of all companies present in the provided datasets (*issuances.csv* and *holdings.csv*), and store them in a dataframe.

References:
- [1]() Introducing the Legal Entity Identifier (LEI) [https://www.gleif.org/en/about-lei/introducing-the-legal-entity-identifier-lei](https://www.gleif.org/en/about-lei/introducing-the-legal-entity-identifier-lei)
- [2]() GLEIF API Introduction [https://documenter.getpostman.com/view/7679680/SVYrrxuU?version=latest#intro](https://documenter.getpostman.com/view/7679680/SVYrrxuU?version=latest#intro)
- [3]() Level 2 Data: Who Owns whom [https://www.gleif.org/en/lei-data/access-and-use-lei-data/level-2-data-who-owns-whom#](https://www.gleif.org/en/lei-data/access-and-use-lei-data/level-2-data-who-owns-whom#)